In [1]:
import os


In [2]:
%pwd

'c:\\Chandu\\WorkSpace\\Learnings\\Repos\\End-to-End-CreditCardFraudDetection-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Chandu\\WorkSpace\\Learnings\\Repos\\End-to-End-CreditCardFraudDetection-Project'

In [19]:
#defining entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path



In [20]:
from creditcard_fraud_detection.constants import *
from creditcard_fraud_detection.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [22]:
import os
import urllib.request as request
import zipfile
from creditcard_fraud_detection.logging import logger
from creditcard_fraud_detection.utils.common import get_size


In [23]:
class DataIngestion:
    def __init__(
            self,
            config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with folling info: \n {headers}")
        else:
            logger.infor(f"File already exists of size: {get_size(Path(self.config.local_data_file))} ")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-26 08:37:06,287: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2025-03-26 08:37:06,293: INFO: common: yaml file: params.yaml loaded succesfully]
[2025-03-26 08:37:06,300: INFO: common: created directory at : artifacts]
[2025-03-26 08:37:06,304: INFO: common: created directory at : artifacts/data_ingestion]
[2025-03-26 08:37:26,963: INFO: 720912943: artifacts/data_ingestion/creditcard.csv.zip download! with folling info: 
 Connection: close
Content-Length: 69155672
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "4a850d178a6a0b428175cd3fa35e37be973bb2ca2907b9652ac55f7e3232125c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7CC6:296D67:191527:3F747D:67E36D83
Accept-Ranges: bytes
Date: Wed, 26 Mar 2025 03:05:41 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100029-HYD